In [1]:
import pandas

In [3]:
ecoli_db = pandas.read_excel('../datasets/datasets_new/databases/ecoli_db.xlsx')

In [6]:
ecoli_dbr = pandas.read_excel('../datasets/datasets_new/databases/ecoli_dbr.xlsx')

In [34]:
ecoli_db.loc[:, 'Ribosomal'] = ecoli_db['Entry'].isin(ecoli_dbr['Entry'].tolist())

In [36]:
ecoli_db['Mass'] = ecoli_db['Mass'].str.replace(',', '').astype(float)

In [38]:
ecoli_db.to_excel('../datasets/datasets_new/databases/ecoli_db_full.xlsx', index = False)

In [85]:
ecoli_comb = ecoli_db.copy()

In [43]:
# ecoli2 processing

In [42]:
ecoli2 = pandas.read_excel('../datasets/datasets_new/ecoli_new/proc/ecoli2_proc.xlsx')

In [44]:
ecoli2.head()

,Protein IDs,Protein names,Gene names,Average copy number per cell
0,P67662,HTH-type transcriptional activator AaeR,aaeR,9.247433
1,P31119,Bifunctional protein aas;2-acylglycerophosphoe...,aas,2.564097
2,P77357,Aminobenzoyl-glutamate utilization protein A,abgA,2.474682
3,P0ABD5,Acetyl-coenzyme A carboxylase carboxyl transfe...,accA,1004.455543
4,P0ABD8,Biotin carboxyl carrier protein of acetyl-CoA ...,accB,624.711430


In [45]:
ecoli2_db_entry_check = ecoli2['Protein IDs'].isin(ecoli_db['Entry'].tolist())

In [47]:
all(ecoli2_db_entry_check)

False

In [139]:
ecoli2_correct = ecoli2[ecoli2_db_entry_check]
ecoli2_tofix = ecoli2[[not x for x in ecoli2_db_entry_check]]

In [52]:
len(ecoli2_correct), len(ecoli2_tofix)

(2265, 5)

In [53]:
ecoli2_tofix

,Protein IDs,Protein names,Gene names,Average copy number per cell
523,P33898,Putative glyceraldehyde-3-phosphate dehydrogen...,gapC,0.983345
749,P00892,Acetolactate synthase isozyme 2 large subunit,ilvG,3.226788
1289,P69407,Capsular synthesis regulator component B,rcsB,488.453982
1290,P14376,Sensor kinase protein RcsC,rcsC,4.311976
1659,P32132,GTP-binding protein typA/BipA,typA,512.366596


In [4]:
def fix_entry_by_gene(genes):
    global ecoli_db
    res = []
    for g in genes:
        for i, j in enumerate(ecoli_db['Gene names'].tolist()):
            if g in str(j):
                if ecoli_db.iloc[i,1] != 'reviewed' or ecoli_db.iloc[i,0] in res: #Filter unreviewed & duplicates
                    continue
                res.append(ecoli_db.iloc[i,0])
                break
        else:
            res.append(0)
                
    return res

In [78]:
ecoli2_tofix.loc[:, 'Protein IDs'] = fix_entry_by_gene(ecoli2_tofix['Gene names'].tolist())

In [81]:
ecoli2_tofix = ecoli2_tofix[[bool(x) for x in ecoli2_tofix['Protein IDs']]]

In [83]:
ecoli2_complete = ecoli2_correct.append(ecoli2_tofix)

In [86]:
ecoli_comb.loc[:, 'ecoli2'] = ecoli_comb['Entry'].map(ecoli2_complete.set_index('Protein IDs')['Average copy number per cell'])

In [90]:
ecoli_comb.head()

,Entry,Status,Protein names,Gene names,Organism,Length,Mass,Gene names (ordered locus ),Entry name,Ribosomal,ecoli2
0,A0A4S5AYA5,unreviewed,Zn(II)/Cd(II)/Pb(II) translocating P-type ATPa...,zntA FAZ83_10695,Escherichia coli (strain K12),732.0,76718.0,NaN,A0A4S5AYA5_ECOLI,False,NaN
1,A0A6D2Y886,unreviewed,Zn-dependent oxidoreductase,FAZ83_03425,Escherichia coli (strain K12),339.0,36534.0,NaN,A0A6D2Y886_ECOLI,False,NaN
2,A0A4S5ATF1,unreviewed,Zn(2+)-responsive transcriptional regulator,zntR FAZ83_23180,Escherichia coli (strain K12),141.0,16195.0,NaN,A0A4S5ATF1_ECOLI,False,NaN
3,P37617,reviewed,Zinc/cadmium/lead-transporting P-type ATPase (...,zntA yhhO b3469 JW3434,Escherichia coli (strain K12),732.0,76840.0,b3469 JW3434,ZNTA_ECOLI,False,40.426732
4,P33030,reviewed,Zinc-binding GTPase YeiR (EC 3.6.-.-),yeiR b2173 JW2161,Escherichia coli (strain K12),328.0,36113.0,b2173 JW2161,YEIR_ECOLI,False,8.999467


In [91]:
#ecoli3 processing

In [92]:
ecoli3 = pandas.read_excel('../datasets/datasets_new/ecoli_new/proc/ecoli3_proc.xlsx')

In [93]:
ecoli3.head()

,Gene name,B number identifier,Avg copies/fl
0,thra,b0002,5560.643127
1,thrb,b0003,947.264345
2,thrc,b0004,7606.583538
3,yaaa,b0006,104.280697
4,talb,b0008,16753.151864


In [100]:
ecoli3.loc[:, 'Gene name'] = ecoli3['Gene name'].map(lambda x: x if len(x) == 3 else x[:3] + x[3].upper())

In [141]:
ecoli3.loc[:, 'Entry'] = fix_entry_by_gene(ecoli3['Gene name'].tolist())

In [132]:
ecoli3 = ecoli3[[bool(x) for x in ecoli3['Entry']]]

In [166]:
ecoli_comb['ecoli3'] = ecoli_comb['Entry'].map(ecoli3.set_index('Entry')['Avg copies/fl'])

In [144]:
#ecoli4 processing

In [145]:
ecoli4 = pandas.read_excel('../datasets/datasets_new/ecoli_new/proc/ecoli4_proc.xlsx')

In [146]:
ecoli4.head()

,Protein IDs,Protein Names,Gene Names,Copies/cell
0,P33937,Periplasmic nitrate reductase,b2206;JW2194;napA;yojC;yojD;yojE,5.135080
1,P0AD65,Penicillin-binding protein 2,b0635;JW0630;mrdA;pbpA,2.304724
2,Q7DFV3,UPF0757 protein ymgG,b1172;JW5178;ymgG,67.419376
3,P77335,"Hemolysin E;Cytotoxin clyA;Hemolysin E, chromo...",ECDH10B_1235;hlyE;b1182;clyA;hpr;JW5181;sheA;ycgD,2.555393
4,P15373,HtrA suppressor protein;Protein prlF,b3129;JW3098;prlF;sohA,8.009775


In [147]:
ecoli4_db_entry_check = ecoli4['Protein IDs'].isin(ecoli_db['Entry'].tolist())

In [148]:
all(ecoli4_db_entry_check)

False

In [155]:
ecoli4_correct = ecoli4[ecoli4_db_entry_check]
ecoli4_tofix = ecoli4[[not x for x in ecoli4_db_entry_check]]

In [156]:
len(ecoli4_correct), len(ecoli4_tofix)

(1997, 5)

In [157]:
ecoli4_tofix

,Protein IDs,Protein Names,Gene Names,Copies/cell
236,P14376,Capsular synthesis regulator component C;Senso...,b2218;JW5917/JW5920;rcsC,114.691484
1024,B1X777;C4ZVY2;P07762,"1,4-alpha-glucan-branching enzyme;1,4-alpha-D-...",ECDH10B_3606;glgB;b3432;JW3395,1426.017787
1338,P69407,Capsular synthesis regulator component B,b2217;JW2205;rcsB,4437.322880
1466,P32132,GTP-binding protein typA/BipA;Tyrosine phospho...,b3871;bipA;JW5571;typA;yihK,8194.209771
1904,P00892,Acetohydroxy-acid synthase II large subunit;Ac...,b3767;b4488;ilvG,0.000000


In [160]:
ecoli4_tofix.loc[:, 'Gene Names'] = ['rcsC', 'glgB', 'rcsB', 'bipA', 'ilvG'] #Manual fix

In [161]:
ecoli4_tofix.loc[:, 'Protein IDs'] = fix_entry_by_gene(ecoli4_tofix['Gene Names'].tolist())

In [162]:
ecoli4_tofix

,Protein IDs,Protein Names,Gene Names,Copies/cell
236,P0DMC5,Capsular synthesis regulator component C;Senso...,rcsC,114.691484
1024,P07762,"1,4-alpha-glucan-branching enzyme;1,4-alpha-D-...",glgB,1426.017787
1338,P0DMC7,Capsular synthesis regulator component B,rcsB,4437.322880
1466,P0DTT0,GTP-binding protein typA/BipA;Tyrosine phospho...,bipA,8194.209771
1904,P0DP89,Acetohydroxy-acid synthase II large subunit;Ac...,ilvG,0.000000


In [163]:
ecoli4_complete = ecoli4_correct.append(ecoli4_tofix)

In [167]:
ecoli_comb.loc[:, 'ecoli4'] = ecoli_comb['Entry'].map(ecoli4_complete.set_index('Protein IDs')['Copies/cell'])

In [168]:
ecoli_comb.head()

,Entry,Status,Protein names,Gene names,Organism,Length,Mass,Gene names (ordered locus ),Entry name,Ribosomal,ecoli2,ecoli3,ecoli4
0,A0A4S5AYA5,unreviewed,Zn(II)/Cd(II)/Pb(II) translocating P-type ATPa...,zntA FAZ83_10695,Escherichia coli (strain K12),732.0,76718.0,NaN,A0A4S5AYA5_ECOLI,False,NaN,NaN,NaN
1,A0A6D2Y886,unreviewed,Zn-dependent oxidoreductase,FAZ83_03425,Escherichia coli (strain K12),339.0,36534.0,NaN,A0A6D2Y886_ECOLI,False,NaN,NaN,NaN
2,A0A4S5ATF1,unreviewed,Zn(2+)-responsive transcriptional regulator,zntR FAZ83_23180,Escherichia coli (strain K12),141.0,16195.0,NaN,A0A4S5ATF1_ECOLI,False,NaN,NaN,NaN
3,P37617,reviewed,Zinc/cadmium/lead-transporting P-type ATPase (...,zntA yhhO b3469 JW3434,Escherichia coli (strain K12),732.0,76840.0,b3469 JW3434,ZNTA_ECOLI,False,40.426732,549.064575,139.518791
4,P33030,reviewed,Zinc-binding GTPase YeiR (EC 3.6.-.-),yeiR b2173 JW2161,Escherichia coli (strain K12),328.0,36113.0,b2173 JW2161,YEIR_ECOLI,False,8.999467,NaN,167.211207


In [169]:
#ecoli5 processing

In [200]:
ecoli5 = pandas.read_excel('../datasets/datasets_new/ecoli_new/proc/ecoli5_proc.xlsx')

In [201]:
ecoli5.head()

,Uniprot Accession,Description,Gene,Copies/cell
0,P0A8T7,DNA-directed RNA polymerase subunit beta' OS=E...,rpoC,2778.519359
1,P0A8V2,DNA-directed RNA polymerase subunit beta OS=Es...,rpoB,3957.434464
2,P36683,Aconitate hydratase 2 OS=Escherichia coli (str...,acnB,7595.542046
3,P15254,Phosphoribosylformylglycinamidine synthase OS=...,purL,2456.265985
4,P09831,Glutamate synthase [NADPH] large chain OS=Esch...,gltB,2858.930632


In [202]:
ecoli5_db_entry_check = ecoli5['Uniprot Accession'].isin(ecoli_db['Entry'].tolist())

In [203]:
all(ecoli5_db_entry_check)

False

In [204]:
ecoli5_correct = ecoli5[ecoli5_db_entry_check]
ecoli5_tofix = ecoli5[[not x for x in ecoli5_db_entry_check]]

In [205]:
len(ecoli5_correct), len(ecoli5_tofix)

(2343, 16)

In [209]:
ecoli5_tofix.loc[:, 'Uniprot Accession'] = fix_entry_by_gene(ecoli5_tofix['Gene'].tolist())

In [210]:
ecoli5_tofix

,Uniprot Accession,Description,Gene,Copies/cell
107,P0DTT0,GTP-binding protein typA/BipA OS=Escherichia c...,typA,3013.027920
416,P75797,GsiB OS=Escherichia coli (strain K12) GN=gsiB ...,gsiB,316.200973
615,P00452,Isoform Alpha' of Ribonucleoside-diphosphate r...,nrdA,28.102454
638,P0DMC7,Capsular synthesis regulator component B OS=Es...,rcsB,2768.759599
917,P78061,Gamma-glutamylputrescine synthetase OS=Escheri...,puuA,122.151966
985,P76038,Gamma-glutamyl-gamma-aminobutyrate hydrolase O...,puuD,113.786338
1232,P50457,Gamma-aminobutyrate aminotransferase OS=Escher...,puuE,41.433804
1390,P75949,Beta-hexosaminidase 2 OS=Escherichia coli (str...,nagZ,130.887279
1544,P02919,Isoform Gamma of Penicillin-binding protein 1B...,mrcB,60.328288
2000,0,ATP-dependent helicase/deoxyribonuclease subun...,addB,12.694666


In [211]:
ecoli5_tofix = ecoli5_tofix[[bool(x) for x in ecoli5_tofix['Uniprot Accession']]]

In [212]:
ecoli5_complete = ecoli5_correct.append(ecoli5_tofix)

In [216]:
ecoli5_complete.drop_duplicates(subset='Uniprot Accession', inplace = True)

In [217]:
ecoli_comb.loc[:, 'ecoli5'] = ecoli_comb['Entry'].map(ecoli5_complete.set_index('Uniprot Accession')['Copies/cell'])

In [218]:
ecoli_comb.head()

,Entry,Status,Protein names,Gene names,Organism,Length,Mass,Gene names (ordered locus ),Entry name,Ribosomal,ecoli2,ecoli3,ecoli4,ecoli5
0,A0A4S5AYA5,unreviewed,Zn(II)/Cd(II)/Pb(II) translocating P-type ATPa...,zntA FAZ83_10695,Escherichia coli (strain K12),732.0,76718.0,NaN,A0A4S5AYA5_ECOLI,False,NaN,NaN,NaN,NaN
1,A0A6D2Y886,unreviewed,Zn-dependent oxidoreductase,FAZ83_03425,Escherichia coli (strain K12),339.0,36534.0,NaN,A0A6D2Y886_ECOLI,False,NaN,NaN,NaN,NaN
2,A0A4S5ATF1,unreviewed,Zn(2+)-responsive transcriptional regulator,zntR FAZ83_23180,Escherichia coli (strain K12),141.0,16195.0,NaN,A0A4S5ATF1_ECOLI,False,NaN,NaN,NaN,NaN
3,P37617,reviewed,Zinc/cadmium/lead-transporting P-type ATPase (...,zntA yhhO b3469 JW3434,Escherichia coli (strain K12),732.0,76840.0,b3469 JW3434,ZNTA_ECOLI,False,40.426732,549.064575,139.518791,11.577466
4,P33030,reviewed,Zinc-binding GTPase YeiR (EC 3.6.-.-),yeiR b2173 JW2161,Escherichia coli (strain K12),328.0,36113.0,b2173 JW2161,YEIR_ECOLI,False,8.999467,NaN,167.211207,54.515815


In [221]:
#ecoli6 processing

In [220]:
ecoli6 = pandas.read_excel('../datasets/datasets_new/ecoli_new/proc/ecoli6_proc.xlsx')

In [222]:
ecoli6.head()

,Uniprot Accession,Description,Gene,Copies/cell
0,P0A8T7,DNA-directed RNA polymerase subunit beta' OS=E...,rpoC,2270.369610
1,P0A8V2,DNA-directed RNA polymerase subunit beta OS=Es...,rpoB,2848.950279
2,P36683,Aconitate hydratase 2 OS=Escherichia coli (str...,acnB,9625.639731
3,P15254,Phosphoribosylformylglycinamidine synthase OS=...,purL,2422.458173
4,P09831,Glutamate synthase [NADPH] large chain OS=Esch...,gltB,1815.060998


In [223]:
ecoli6_db_entry_check = ecoli6['Uniprot Accession'].isin(ecoli_db['Entry'].tolist())

In [224]:
ecoli6_correct = ecoli6[ecoli6_db_entry_check]
ecoli6_tofix = ecoli6[[not x for x in ecoli6_db_entry_check]]

In [225]:
len(ecoli6_correct), len(ecoli6_tofix)

(2007, 12)

In [227]:
ecoli6_tofix.loc[:, 'Uniprot Accession'] = fix_entry_by_gene(ecoli6_tofix['Gene'].tolist())

In [228]:
ecoli6_tofix

,Uniprot Accession,Description,Gene,Copies/cell
109,P0DTT0,GTP-binding protein typA/BipA OS=Escherichia c...,typA,2934.827123
526,P0DMC7,Capsular synthesis regulator component B OS=Es...,rcsB,1971.891617
839,P0DP89,Acetolactate synthase isozyme 2 large subunit ...,ilvG,63.487444
935,P75949,Beta-hexosaminidase 2 OS=Escherichia coli (str...,nagZ,117.005929
951,P78061,Gamma-glutamylputrescine synthetase OS=Escheri...,puuA,113.735710
1060,P75797,GsiB OS=Escherichia coli (strain K12) GN=gsiB ...,gsiB,189.690381
1581,P76038,Gamma-glutamyl-gamma-aminobutyrate hydrolase O...,puuD,66.573286
1733,Q46896,CRISPR-associated protein cas1 OS=Lactobacillu...,cas1,2240.872673
1765,P42641,GTPase Obg/CgtA OS=Vibrio harveyi GN=cgtA PE=1...,cgtA,61.110373
1824,0,Similarity to HYPOTHETICAL METHYLTRANSFERASE-L...,ECU1,16.473276


In [229]:
ecoli6_tofix = ecoli6_tofix[[bool(x) for x in ecoli6_tofix['Uniprot Accession']]]

In [230]:
ecoli6_complete = ecoli6_correct.append(ecoli6_tofix)

In [231]:
ecoli6_complete.drop_duplicates(subset='Uniprot Accession', inplace = True)

In [232]:
ecoli_comb.loc[:, 'ecoli6'] = ecoli_comb['Entry'].map(ecoli6_complete.set_index('Uniprot Accession')['Copies/cell'])

In [233]:
ecoli_comb.head()

,Entry,Status,Protein names,Gene names,Organism,Length,Mass,Gene names (ordered locus ),Entry name,Ribosomal,ecoli2,ecoli3,ecoli4,ecoli5,ecoli6
0,A0A4S5AYA5,unreviewed,Zn(II)/Cd(II)/Pb(II) translocating P-type ATPa...,zntA FAZ83_10695,Escherichia coli (strain K12),732.0,76718.0,NaN,A0A4S5AYA5_ECOLI,False,NaN,NaN,NaN,NaN,NaN
1,A0A6D2Y886,unreviewed,Zn-dependent oxidoreductase,FAZ83_03425,Escherichia coli (strain K12),339.0,36534.0,NaN,A0A6D2Y886_ECOLI,False,NaN,NaN,NaN,NaN,NaN
2,A0A4S5ATF1,unreviewed,Zn(2+)-responsive transcriptional regulator,zntR FAZ83_23180,Escherichia coli (strain K12),141.0,16195.0,NaN,A0A4S5ATF1_ECOLI,False,NaN,NaN,NaN,NaN,NaN
3,P37617,reviewed,Zinc/cadmium/lead-transporting P-type ATPase (...,zntA yhhO b3469 JW3434,Escherichia coli (strain K12),732.0,76840.0,b3469 JW3434,ZNTA_ECOLI,False,40.426732,549.064575,139.518791,11.577466,15.682215
4,P33030,reviewed,Zinc-binding GTPase YeiR (EC 3.6.-.-),yeiR b2173 JW2161,Escherichia coli (strain K12),328.0,36113.0,b2173 JW2161,YEIR_ECOLI,False,8.999467,NaN,167.211207,54.515815,96.054656


In [234]:
#ecoli7 processing

In [235]:
ecoli7 = pandas.read_excel('../datasets/datasets_new/ecoli_new/proc/ecoli7_proc.xlsx')

In [236]:
ecoli7.head()

,Uniprot Accession,Description,Gene,Copies/cell
0,P0A8T7,DNA-directed RNA polymerase subunit beta' OS=E...,rpoC,5878.961980
1,P0A8V2,DNA-directed RNA polymerase subunit beta OS=Es...,rpoB,6028.691561
2,P36683,Aconitate hydratase 2 OS=Escherichia coli (str...,acnB,10998.651286
3,P15254,Phosphoribosylformylglycinamidine synthase OS=...,purL,3523.265533
4,P09831,Glutamate synthase [NADPH] large chain OS=Esch...,gltB,3561.651804


In [237]:
ecoli7_db_entry_check = ecoli7['Uniprot Accession'].isin(ecoli_db['Entry'].tolist())

In [238]:
ecoli7_correct = ecoli7[ecoli7_db_entry_check]
ecoli7_tofix = ecoli7[[not x for x in ecoli7_db_entry_check]]

In [239]:
len(ecoli7_correct), len(ecoli7_tofix)

(2007, 12)

In [241]:
ecoli7_tofix.loc[:, 'Uniprot Accession'] = fix_entry_by_gene(ecoli7_tofix['Gene'].tolist())

In [242]:
ecoli7_tofix = ecoli7_tofix[[bool(x) for x in ecoli7_tofix['Uniprot Accession']]]

In [243]:
ecoli7_complete = ecoli7_correct.append(ecoli7_tofix)

In [244]:
ecoli7_complete.drop_duplicates(subset='Uniprot Accession', inplace = True)

In [245]:
ecoli_comb.loc[:, 'ecoli7'] = ecoli_comb['Entry'].map(ecoli7_complete.set_index('Uniprot Accession')['Copies/cell'])

In [246]:
ecoli_comb.head()

,Entry,Status,Protein names,Gene names,Organism,Length,Mass,Gene names (ordered locus ),Entry name,Ribosomal,ecoli2,ecoli3,ecoli4,ecoli5,ecoli6,ecoli7
0,A0A4S5AYA5,unreviewed,Zn(II)/Cd(II)/Pb(II) translocating P-type ATPa...,zntA FAZ83_10695,Escherichia coli (strain K12),732.0,76718.0,NaN,A0A4S5AYA5_ECOLI,False,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A6D2Y886,unreviewed,Zn-dependent oxidoreductase,FAZ83_03425,Escherichia coli (strain K12),339.0,36534.0,NaN,A0A6D2Y886_ECOLI,False,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A4S5ATF1,unreviewed,Zn(2+)-responsive transcriptional regulator,zntR FAZ83_23180,Escherichia coli (strain K12),141.0,16195.0,NaN,A0A4S5ATF1_ECOLI,False,NaN,NaN,NaN,NaN,NaN,NaN
3,P37617,reviewed,Zinc/cadmium/lead-transporting P-type ATPase (...,zntA yhhO b3469 JW3434,Escherichia coli (strain K12),732.0,76840.0,b3469 JW3434,ZNTA_ECOLI,False,40.426732,549.064575,139.518791,11.577466,15.682215,14.692005
4,P33030,reviewed,Zinc-binding GTPase YeiR (EC 3.6.-.-),yeiR b2173 JW2161,Escherichia coli (strain K12),328.0,36113.0,b2173 JW2161,YEIR_ECOLI,False,8.999467,NaN,167.211207,54.515815,96.054656,160.269489


In [271]:
#ecoli9 processing

In [272]:
ecoli9 = pandas.read_excel('../datasets/datasets_new/ecoli_new/proc/ecoli9_proc.xlsx') #MG1655

In [273]:
ecoli9.head()

,Gene name,Gene locus,Protein ID,Protein mass fractions
0,aaeA,b3241,P46482,0.000000
1,aaeB,b3240,P46481,0.000000
2,aaeR,b3243,P67662,0.000000
3,aaeX,b3242,P46478,0.000000
4,aas,b2836,P31119,0.000046


In [274]:
ecoli9_db_entry_check = ecoli9['Protein ID'].isin(ecoli_db['Entry'].tolist())

In [275]:
all(ecoli9_db_entry_check)

False

In [276]:
ecoli9_correct = ecoli9[ecoli9_db_entry_check]
ecoli9_tofix = ecoli9[[not x for x in ecoli9_db_entry_check]]

In [277]:
len(ecoli9_correct), len(ecoli9_tofix)

(4211, 131)

In [278]:
ecoli9_tofix.dropna(inplace = True)

C:\Users\Dynev\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [279]:
ecoli9_tofix

,Gene name,Gene locus,Protein ID,Protein mass fractions
10,abpA,b2628,P52137,0.000000
262,bipA,b3871,P32132,0.001833
929,gapC_1,b1417,P33898,0.000047
1263,ilvG_1,b3767,P00892,0.000578
1411,ldrA,b4419,Q6BF86,0.000000
1413,ldrC,b4423,Q6BF86,0.000000
3035,ybeM_1,b0625,P39874,0.000000
3052,ybfK,b4590,P46121,0.000000
3412,yedM,b1935,P76321,0.000000
3709,ygiA,b3036,Q79CP2,0.000000


In [280]:
ecoli9_tofix.loc[:, 'Protein ID'] = fix_entry_by_gene(ecoli9_tofix['Gene name'].tolist())

C:\Users\Dynev\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [281]:
ecoli9_tofix = ecoli9_tofix[[bool(x) for x in ecoli9_tofix['Protein ID']]]

In [282]:
ecoli9_tofix

,Gene name,Gene locus,Protein ID,Protein mass fractions
262,bipA,b3871,P0DTT0,0.001833
1411,ldrA,b4419,P0DPD0,0.000000
1413,ldrC,b4423,P0DPD1,0.000000
3412,yedM,b1935,P76322,0.000000
4018,yjiP,b4338,P0DP21,0.000000


In [283]:
ecoli9_complete = ecoli9_correct.append(ecoli9_tofix)

In [284]:
ecoli9_complete.drop_duplicates(subset='Protein ID', inplace = True)

In [285]:
ecoli_comb.loc[:, 'ecoli9'] = ecoli_comb['Entry'].map(ecoli9_complete.set_index('Protein ID')['Protein mass fractions'])

In [286]:
ecoli_comb

,Entry,Status,Protein names,Gene names,Organism,Length,Mass,Gene names (ordered locus ),Entry name,Ribosomal,ecoli2,ecoli3,ecoli4,ecoli5,ecoli6,ecoli7,ecoli8,ecoli9
0,A0A4S5AYA5,unreviewed,Zn(II)/Cd(II)/Pb(II) translocating P-type ATPa...,zntA FAZ83_10695,Escherichia coli (strain K12),732.0,76718.0,NaN,A0A4S5AYA5_ECOLI,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0A6D2Y886,unreviewed,Zn-dependent oxidoreductase,FAZ83_03425,Escherichia coli (strain K12),339.0,36534.0,NaN,A0A6D2Y886_ECOLI,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A4S5ATF1,unreviewed,Zn(2+)-responsive transcriptional regulator,zntR FAZ83_23180,Escherichia coli (strain K12),141.0,16195.0,NaN,A0A4S5ATF1_ECOLI,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P37617,reviewed,Zinc/cadmium/lead-transporting P-type ATPase (...,zntA yhhO b3469 JW3434,Escherichia coli (strain K12),732.0,76840.0,b3469 JW3434,ZNTA_ECOLI,False,40.426732,549.064575,139.518791,11.577466,15.682215,14.692005,0.000009,0.000010
4,P33030,reviewed,Zinc-binding GTPase YeiR (EC 3.6.-.-),yeiR b2173 JW2161,Escherichia coli (strain K12),328.0,36113.0,b2173 JW2161,YEIR_ECOLI,False,8.999467,NaN,167.211207,54.515815,96.054656,160.269489,0.000039,0.000042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9476,P76078,reviewed,"1,2-phenylacetyl-CoA epoxidase, subunit B (1,2...",paaB ynbF b1389 JW1384,Escherichia coli (strain K12),95.0,10942.0,b1389 JW1384,PAAB_ECOLI,False,NaN,NaN,NaN,0.024633,0.011180,0.294185,0.000000,0.000000
9477,P76077,reviewed,"1,2-phenylacetyl-CoA epoxidase, subunit A (EC ...",paaA ydbO b1388 JW1383,Escherichia coli (strain K12),309.0,35499.0,b1388 JW1383,PAAA_ECOLI,False,NaN,NaN,NaN,NaN,0.005077,0.015905,0.000000,0.000000
9478,P77467,reviewed,"1,2-epoxyphenylacetyl-CoA isomerase (EC 5.3.3.18)",paaG ydbT b1394 JW1389,Escherichia coli (strain K12),262.0,28405.0,b1394 JW1389,PAAG_ECOLI,False,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
9479,A0A6D2XEG8,unreviewed,10 kDa chaperonin (GroES protein) (Protein Cpn10),groES groS FAZ83_09765,Escherichia coli (strain K12),97.0,10387.0,NaN,A0A6D2XEG8_ECOLI,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [287]:
#ecoli9 processing

In [288]:
ecoli9 = pandas.read_excel('../datasets/datasets_new/ecoli_new/proc/ecoli9_proc.xlsx') #MG1655

In [289]:
ecoli9.head()

,Gene name,Gene locus,Protein ID,Protein mass fractions
0,aaeA,b3241,P46482,0.000000
1,aaeB,b3240,P46481,0.000000
2,aaeR,b3243,P67662,0.000000
3,aaeX,b3242,P46478,0.000000
4,aas,b2836,P31119,0.000046


In [290]:
ecoli9_db_entry_check = ecoli9['Protein ID'].isin(ecoli_db['Entry'].tolist())

In [291]:
all(ecoli9_db_entry_check)

False

In [292]:
ecoli9_correct = ecoli9[ecoli9_db_entry_check]
ecoli9_tofix = ecoli9[[not x for x in ecoli9_db_entry_check]]

In [293]:
len(ecoli9_correct), len(ecoli9_tofix)

(4211, 131)

In [294]:
ecoli9_tofix.dropna(inplace = True)

C:\Users\Dynev\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [295]:
ecoli9_tofix

,Gene name,Gene locus,Protein ID,Protein mass fractions
10,abpA,b2628,P52137,0.000000
262,bipA,b3871,P32132,0.001833
929,gapC_1,b1417,P33898,0.000047
1263,ilvG_1,b3767,P00892,0.000578
1411,ldrA,b4419,Q6BF86,0.000000
1413,ldrC,b4423,Q6BF86,0.000000
3035,ybeM_1,b0625,P39874,0.000000
3052,ybfK,b4590,P46121,0.000000
3412,yedM,b1935,P76321,0.000000
3709,ygiA,b3036,Q79CP2,0.000000


In [296]:
ecoli9_tofix.loc[:, 'Protein ID'] = fix_entry_by_gene(ecoli9_tofix['Gene name'].tolist())

C:\Users\Dynev\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [297]:
ecoli9_tofix = ecoli9_tofix[[bool(x) for x in ecoli9_tofix['Protein ID']]]

In [298]:
ecoli9_tofix

,Gene name,Gene locus,Protein ID,Protein mass fractions
262,bipA,b3871,P0DTT0,0.001833
1411,ldrA,b4419,P0DPD0,0.000000
1413,ldrC,b4423,P0DPD1,0.000000
3412,yedM,b1935,P76322,0.000000
4018,yjiP,b4338,P0DP21,0.000000


In [299]:
ecoli9_complete = ecoli9_correct.append(ecoli9_tofix)

In [300]:
ecoli9_complete.drop_duplicates(subset='Protein ID', inplace = True)

In [301]:
ecoli_comb.loc[:, 'ecoli9'] = ecoli_comb['Entry'].map(ecoli9_complete.set_index('Protein ID')['Protein mass fractions'])

In [303]:
ecoli_comb.to_excel('../datasets/datasets_new/ecoli_new/ecoli_comb_og.xlsx', index = False)